# West Africa 1990-2021
#### Download Observation data

J.Joel & Dr.Olugboji  URSeismo   02/2022 modified from MTtime 1.1.0 (Accessible on Github: https://github.com/LLNL/mttime/tree/master/examples/notebooks)

In [1]:
import os
from pathlib import Path
from obspy.clients.fdsn import Client
from obspy import read_events, UTCDateTime
from obspy.clients.fdsn.mass_downloader import CircularDomain, Restrictions, MassDownloader
import pandas as pd
import sys

In [2]:
!pip show obspy

Name: obspy
Version: 1.2.2
Summary: ObsPy - a Python framework for seismological observatories.
Home-page: https://www.obspy.org
Author: The ObsPy Development Team
Author-email: devs@obspy.org
License: GNU Lesser General Public License, Version 3 (LGPLv3)
Location: /gpfs/fs2/scratch/tolugboj_lab/softwares/anaconda/anaconda3/2021.05/lib/python3.8/site-packages
Requires: decorator, future, lxml, matplotlib, numpy, requests, scipy, setuptools, sqlalchemy
Required-by: instaseis, mqs-reports, mttime


#### Create Catalog of event

In [ ]:
event_bool = True

if event_bool:
    dataCenter="IRIS"
    client = Client(dataCenter)
    
    starttime = UTCDateTime("1990-01-01T00:00:00")
    endtime = UTCDateTime("2021-12-31T23:59:59")
    
    
    catalog = client.get_events(starttime=starttime, endtime=endtime,
                        minmagnitude=2.0,maxmagnitude=5.0,
                        minlatitude=4.58, maxlatitude=24.78,
                        minlongitude= -17.33, maxlongitude=23.98,
                        includeallorigins=True)
    
   
    
    Path(quakemldir).mkdir(parents=True,exist_ok=True)

    #Write earthquakes catalog
    catalog.write(("%s/WafrQuakes90_21.xml")%quakemldir,format="QUAKEML") 
    

In [ ]:
catalog = read_events("quakes_xml/WafrQuakes90_21.xml")
for i, event in enumerate(catalog[1:]):
    print(str(i) + "_" + str(event.origins[0].time))
catalog.plot(projection="local");

#### Download Observation data from IRIS


In [ ]:
dataCenter="IRIS" 

# Time before and after event origin for waveform segments
time_before = 5*60 #seconds
time_after = 60*60 #seconds
download_bool = True


catalog = read_events("quakes_xml/WafrQuakes90_21.xml")

for event in catalog:
    try:
        evid = str(event.origins[0].time)
        evid = evid[0:4] + evid[5:7] + evid[8:13] + evid[14:16] + evid[17:19]
        evloc = event.event_descriptions[0].text
        evid = evloc[:4] + evid
        outdir = ("data/events/%s")%evid


        Path(outdir).mkdir(parents=True,exist_ok=True)


        # Event origin
        origin_time = event.preferred_origin().time
        starttime = origin_time - time_before
        endtime = origin_time + time_after

        # Event location
        evlo = event.preferred_origin().longitude
        evla = event.preferred_origin().latitude
        depth = event.preferred_origin().depth # in meters

        # Set the search area
        domain = CircularDomain(latitude=evla, longitude=evlo, minradius=0, maxradius=60)

        # Set the search period and additional criteria
        restrictions = Restrictions(starttime=starttime, endtime=endtime,
            reject_channels_with_gaps=True,
            minimum_length=0.95,
            network="G,GE,II,KZ,XB,XJ,XK,DK,GT,IU,AF,4H,5H,6H,MN,BX,CM,PL,AC,CZ,KR,NR,RO,XW,NC,IU,NJ,AF,GH,XD",
            station="TAM,ABKT,ABKAR,PM29,PM34,PM35,LN46,B13NX,B14MH,ICESG,DBIC,KOWA,KUKU,SHAI,EDA,SOK,NAB1,NAB2,NAB3,NAB4,NAB5,NAB8,FAME,BANH,TUE,PM26,B01KR,B03SL,B15MW,Q04NM,Q06MQ,W04VR,GRTLG,RUS,MBAR,NIE,SACV,VLO,SVMA,MBO,DPC,BTK,NRN,TRKS,NE207,BUR31,KITG,LIRA,IRGVP,MSKU,BNG,TORO,KAD,WEIJ,BGCA,GOMA,FURI",
            channel_priorities=["BH[ZNE12]"],                       
            sanitize=True)

        # Save catalog info to file
        event_out = (
            "{evid:s},{origin:s},{jdate:s},"
            "{lon:.4f},{lat:.4f},{depth:.4f},"
            "{mag:.2f},{auth:s}\n"
            )        

        if event.preferred_magnitude() is None:
            mag = -999.
            magtype = "ml"
        else:
            mag = event.preferred_magnitude().mag
            magtype = event.preferred_magnitude().magnitude_type
        if event.preferred_origin().extra.catalog.value is None:
            auth = "unknown"
        else:
            auth = event.preferred_origin().extra.catalog.value.replace(" ","")

        event_out = event_out.format(
            evid=evid,
            origin=str(origin_time),
            jdate="%s%s"%(origin_time.year,origin_time.julday),
            lon=evlo,
            lat=evla,
            depth=depth/1E3,
            mag=mag,
            auth=auth
            )

        outfile = ("data/events/%s/datetime.csv")%evid
        with open(outfile,"w") as f:
            f.write("evid,origin,jdate,lon,lat,depth,%s,auth\n"%magtype)
            f.write(event_out)

        # Dowanload waveforms and metadata
        if download_bool:
            mseed_storage = "%s/waveforms"%outdir
            stationxml_storage = "%s/stations"%outdir
            mdl = MassDownloader(providers=[dataCenter])
            mdl_helper = mdl.download(domain, restrictions,
                mseed_storage=mseed_storage,stationxml_storage=stationxml_storage)
            print("%s download completed"%outdir)


        print("%s is DONE."%outdir)
        
    except Exception as e:
                print("there was an exception %s", str(e))
                exception_type, exception_object, exception_traceback = sys.exc_info()
                filename = exception_traceback.tb_frame.f_code.co_filename
                line_number = exception_traceback.tb_lineno

                print("Exception type: ", exception_type)
                print("File name: ", filename)
                print("Line number: ", line_number)

##### GE.ACRG

In [ ]:
dataCenter="http://geofon.gfz-potsdam.de" 

# Time before and after event origin for waveform segments
time_before = 5*60 #seconds
time_after = 60*60 #seconds
download_bool = True

catalog = read_events("quakes_xml/WafrQuakes90_21.xml")

for event in catalog:
    try:
        evid = str(event.origins[0].time)
        evid = evid[0:4] + evid[5:7] + evid[8:13] + evid[14:16] + evid[17:19]
        evloc = event.event_descriptions[0].text
        evid = evloc[:4] + evid
        outdir = ("data/events/%s")%evid

        Path(outdir).mkdir(parents=True,exist_ok=True)


        # Event origin
        origin_time = event.preferred_origin().time
        starttime = origin_time - time_before
        endtime = origin_time + time_after

        # Event location
        evlo = event.preferred_origin().longitude
        evla = event.preferred_origin().latitude
        depth = event.preferred_origin().depth # in meters

        # Set the search area
        domain = CircularDomain(latitude=evla, longitude=evlo, minradius=0, maxradius=60)

        # Set the search period and additional criteria
        restrictions = Restrictions(starttime=starttime, endtime=endtime,
            reject_channels_with_gaps=True,
            minimum_length=0.95,
            network="GE",
            station="ACRG",
            channel_priorities=["BH[ZNE12]"],                       
            sanitize=True)

        # Save catalog info to file
        event_out = (
            "{evid:s},{origin:s},{jdate:s},"
            "{lon:.4f},{lat:.4f},{depth:.4f},"
            "{mag:.2f},{auth:s}\n"
            )        

        if event.preferred_magnitude() is None:
            mag = -999.
            magtype = "ml"
        else:
            mag = event.preferred_magnitude().mag
            magtype = event.preferred_magnitude().magnitude_type.lower()
        if event.preferred_origin().extra.catalog.value is None:
            auth = "unknown"
        else:
            auth = event.preferred_origin().extra.catalog.value.replace(" ","")

        event_out = event_out.format(
            evid=evid,
            origin=str(origin_time),
            jdate="%s%s"%(origin_time.year,origin_time.julday),
            lon=evlo,
            lat=evla,
            depth=depth/1E3,
            mag=mag,
            auth=auth
            )
        
        # Dowanload waveforms and metadata
        if download_bool:
            mseed_storage = "%s/waveforms"%outdir
            stationxml_storage = "%s/stations"%outdir
            mdl = MassDownloader(providers=[dataCenter])
            mdl_helper = mdl.download(domain, restrictions,
                mseed_storage=mseed_storage,stationxml_storage=stationxml_storage)
            print("%s download completed"%outdir)


        print("%s is DONE."%outdir)
        
    except Exception as e:
                print("there was an exception %s", str(e))
                exception_type, exception_object, exception_traceback = sys.exc_info()
                filename = exception_traceback.tb_frame.f_code.co_filename
                line_number = exception_traceback.tb_lineno

                print("Exception type: ", exception_type)
                print("File name: ", filename)
                print("Line number: ", line_number)